Perhaps we should perform some clustering of products before running regression? 

In [26]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

dff = pd.read_csv('merged_df.csv')
df = dff[['country','product','rca','price','labour','capital']].copy()
product_dict_df = pd.read_csv('BACI_HS17_V202401/product_codes_HS17_V202401.csv')
product_dict = dict(zip(product_dict_df['code'], product_dict_df['description']))

In [27]:
# check for missing values
missing_values = df.isnull().sum()
print("Missing values per column:\n", missing_values)
df_clean = df.dropna()

Missing values per column:
 country    0
product    0
rca        0
price      0
labour     0
capital    0
dtype: int64


In [13]:
# Normalising the explanatory variables
# Select independent variables
independent_vars = ['price', 'labour', 'capital']

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the independent variables
df_clean[independent_vars] = scaler.fit_transform(df_clean[independent_vars])

In [42]:
def analyze_product(df, product_code):
    """
    Perform linear regression on a specific product code.
    
    Args:
    - df (DataFrame): Input DataFrame containing the relevant product data.
    - product_code (str or int): HS code of the product to analyze.
    
    Returns:
    - (float): Coefficient of the energy price
    """
    # Filter the data to include only the specified product
    product_data = df[df['product'] == product_code]
    
    # Set dependent and independent variables
    X = product_data[['price', 'labour', 'capital']]
    y = product_data['rca']
    
    # Add a constant to the model (for the intercept)
    X = sm.add_constant(X)
    
    # Fit the model
    model = sm.OLS(y, X).fit()
    
    # Return the coefficient of energy price ('price')
    return model.params['price']

In [43]:
# Get the unique product codes
unique_products = df_clean['product'].unique()

# Create a dictionary to store coefficients
price_coefficients = {}

# Analyze each product
for product_code in unique_products:
    coefficient = analyze_product(df_clean, product_code)
    price_coefficients[product_code] = coefficient

# Convert to a DataFrame for ranking
coeff_df = pd.DataFrame.from_dict(price_coefficients, orient='index', columns=['price_coefficient'])
coeff_df.index.name = 'product'
coeff_df.reset_index(inplace=True)

# Rank products by price sensitivity
coeff_df = coeff_df.sort_values(by='price_coefficient', ascending=False)

In [44]:
coeff_df['product_desc'] = coeff_df['product'].map(product_dict)

In [36]:
coeff_df.to_csv('OLS.csv')

In [45]:
coeff_df

,product,price_coefficient,product_desc
3482,854012,155.604816,"Tubes: cathode-ray television picture tubes, i..."
410,90710,132.184003,"Spices: cloves (whole fruit, cloves and stems)..."
1171,71440,126.793113,Vegetable roots and tubers: taro (Colocasia sp...
4323,120300,120.798451,Copra
5113,30784,98.917434,"Molluscs: stromboid conchs (Strombus spp.), wh..."
...,...,...,...
5111,30292,-49.622482,"Fish: fresh or chilled, shark fins"
3608,251810,-56.946503,"Dolomite, not calcined or sintered: including ..."
5104,261220,-60.843827,Thorium ores and concentrates
904,500100,-69.887780,Silk: silk-worm cocoons suitable for reeling
